In [1]:
%load_ext autoreload
%autoreload 2
from manage_orderbooks import *
from orderbook_trade_simulator import *
from IPython.display import display
%matplotlib inline

In [2]:
filename = '../data/snapshots/orderbooks_USDT_BTC_range1.15_samplesNone_snapshot2016-12-15T15:09.dict'
orderbooks, timestamps = load_orderbook_snapshot(infile=filename, first_line=0, last_line=250)
# display(timestamps)
print(len(orderbooks), len(orderbooks[0]))

100%|██████████| 250/250 [00:03<00:00, 69.27it/s]


Loaded Orderbooks: 250
(250, 908)


In [39]:
def buy(df, volume, limit=None):
    # buy from market
    df = df[df.Type == 'ask'].copy()
    
    print("  volume: {}".format(volume))
    print("  limit : {}".format(limit))
    
    cashflow = 0
    volume_traded = 0

    for pos in range(len(df)):
        order = df.iloc[pos]
        price = str(order.Price)

        if (limit and order.Price > limit):
            # Price limit exceeded, stop trading now!
            break

        if volume - order.Amount >= 0:
            current_order_volume = order.Amount
        else:
            current_order_volume = volume

        cashflow += current_order_volume * order.Price
        volume_traded += current_order_volume

        volume -= current_order_volume
        
        if round(volume, 10) == 0.:
            break
            
    avg_price = np.nan
    if volume_traded != 0:
        avg_price = cashflow / volume_traded
    
    return cashflow, round(volume_traded, 8), avg_price

def discretize_inventar(x, base, Type='trunc'):
    assert isinstance(x, float) or isinstance(x, int)
    assert isinstance(base, float)
    assert isinstance(Type,str) and (Type=='trunc' or Type=='round')
    
    if Type=='trunc':
        return (np.trunc(x / base)) * base
    elif Type=='round':
        return round(x / base) * base

def discretize_inventar_round(x, steps):
    inv = np.trunc(x * steps)
    
    return inv

print(discretize_inventar_round(1, 4))
print("")

# print(discretize_inventar(0.00, base=0.25))
# print(discretize_inventar(0.24, base=0.25))
# print(discretize_inventar(0.25, base=0.25))
# print(discretize_inventar(0.49, base=0.25))
# print(discretize_inventar(0.5, base=0.25))
# print(discretize_inventar(0.75, base=0.25))
# print(discretize_inventar(0.95, base=0.25))
# print(discretize_inventar(1., base=0.25))

def transform_state(t, i, I=4):
    return t*I + i

4.0



In [40]:
x = 0.9
base = 0.25
print((np.trunc(x / base)) * base)

ev = round(x / base) * base
ev2 = (np.trunc(x / base)) * base

print(ev, ev2)

0.75
(1.0, 0.75)


In [63]:
trade_volume = 10.  # shares
trade_time = 20  # min
A = [-.7]  # available actions

T = 5 # time horizon
I = 4  # remaining inventory rounded
L = len(A)  # number of actions

vol_interval = [(i+1.) / I for i in range(I)]
print(vol_interval)
NUM_STATES = I * T
NUM_ACTIONS = L
state_names = []
for t in range(T):
    for i in range(I):
        state_names.append('(t{}, i{})'.format(t, i))
        



Q = pd.DataFrame(np.zeros([T*I, L]), index=state_names)
# display(Q)
n=0
for experiment in range(1):
    orderbooks_window = [orderbooks[10*experiment+idx] for idx in range(T)]
    initial_center = (orderbooks_window[0])[orderbooks_window[0].Type == 'center'].Price.values[0]
    baseline_price = initial_center * trade_volume
    print("init: {}, baseline_price: {}".format(initial_center, baseline_price))

    print("len", len(orderbooks_window))
    for t in range(T)[::-1]:
        print("t", t)

        book = orderbook_enrich(orderbooks_window[t])
        current_center = book[book.Type == 'center'].Price.values[0]
        current_ask = book[book.Type == 'ask'].Price.values[0]

        display(orderbook_preview(book, 3))
        # plot_orderbook(book, title=timestamps_window[t], range_factor=1.005)

        for i, vol in enumerate(vol_interval):
            state = transform_state(t, i)
            print("vol", vol, i)
            
            for ai, a in enumerate(A):

                lim = current_ask - a
                print("action: {}, lim={}".format(a, lim))
                
                if t == T-1:
                    # last round, place market order (buy without limit)
                    # lim = None
                    pass

                # place order
                cashflow, volume_traded, avg_price = buy(df=book, volume=vol*trade_volume, limit=lim)
                volume_traded_percent = volume_traded / trade_volume
                print("volume_traded_percent: {} ({}/{}) {}".format(volume_traded_percent, volume_traded, trade_volume, volume_traded / vol))
                
                # check results
                cost = cashflow - volume_traded * current_center
                print("cost", cost, cost/volume_traded, cashflow, volume_traded * current_center)
                
                new_vol = round_shares(vol-volume_traded_percent, 1./I) + 1./I
                new_vol2 = discretize_inventar(vol-volume_traded, 1./I)
                print("new_vol", new_vol, vol-volume_traded_percent, vol-(volume_traded/I), new_vol2)
                print("cashflow", volume_traded, cashflow, vol, new_vol)
                print(new_vol)
                new_state = Q.iloc[transform_state(t, vol_interval.index(new_vol))]

                Q.iloc[state, ai] = n/(n+1) * Q.iloc[state, ai] + 1./(n+1) * (cost + Q.iloc[state, ai])

        n+=1

    display(pd.DataFrame(Q))

[0.25, 0.5, 0.75, 1.0]
init: 705.224976072, baseline_price: 7052.24976072
('len', 5)
('t', 4)


,Amount,Price,Type,norm_Price,Volume,VolumeAcc
988,1.700000,703.360000,bid,0.998205,1195.712000,4908.078940
989,0.001250,703.700000,bid,0.998688,0.879625,3712.366940
990,5.273497,703.800000,bid,0.998830,3711.487315,3711.487315
991,0.000000,704.624678,center,1.000000,0.000000,0.000000
992,6.842379,705.450000,ask,1.001171,4826.956103,4826.956103
993,0.052838,706.170000,ask,1.002193,37.312497,4864.268601
994,0.645323,706.190000,ask,1.002221,455.720409,5319.989010


('vol', 0.25, 0)
action: -0.7, lim=706.15
  volume: 2.5
  limit : 706.15
volume_traded_percent: 0.25 (2.5/10.0) 10.0
('cost', 2.0633049498956098, 0.82532197995824386, 1763.625, 1761.5616950501044)
('new_vol', 0.25, 0.0, -0.375, -2.25)
('cashflow', 2.5, 1763.625, 0.25, 0.25)
0.25
('vol', 0.5, 1)
action: -0.7, lim=706.15
  volume: 5.0
  limit : 706.15
volume_traded_percent: 0.5 (5.0/10.0) 10.0
('cost', 4.1266098997912195, 0.82532197995824386, 3527.25, 3523.1233901002088)
('new_vol', 0.25, 0.0, -0.75, -4.5)
('cashflow', 5.0, 3527.25, 0.5, 0.25)
0.25
('vol', 0.75, 2)
action: -0.7, lim=706.15
  volume: 7.5
  limit : 706.15
volume_traded_percent: 0.684237877 (6.84237877/10.0) 9.12317169333
('cost', 5.6471655940813434, 0.82532197995834478, 4826.9561032965003, 4821.3089377024189)
('new_vol', 0.25, 0.06576212300000006, -0.9605946925, -6.0)
('cashflow', 6.84237877, 4826.9561032965003, 0.75, 0.25)
0.25
('vol', 1.0, 3)
action: -0.7, lim=706.15
  volume: 10.0
  limit : 706.15
volume_traded_percent:

,Amount,Price,Type,norm_Price,Volume,VolumeAcc
988,1.700000,703.360000,bid,0.998205,1195.712000,4910.381485
989,0.001250,703.700000,bid,0.998688,0.879625,3714.669485
990,5.276769,703.800000,bid,0.998830,3713.789860,3713.789860
991,0.000000,704.624678,center,1.000000,0.000000,0.000000
992,5.088456,705.450000,ask,1.001171,3589.651144,3589.651144
993,0.052838,706.170000,ask,1.002193,37.312497,3626.963642
994,0.645323,706.190000,ask,1.002221,455.720409,4082.684051


('vol', 0.25, 0)
action: -0.7, lim=706.15
  volume: 2.5
  limit : 706.15
volume_traded_percent: 0.25 (2.5/10.0) 10.0
('cost', 2.0633049498956098, 0.82532197995824386, 1763.625, 1761.5616950501044)
('new_vol', 0.25, 0.0, -0.375, -2.25)
('cashflow', 2.5, 1763.625, 0.25, 0.25)
0.25
('vol', 0.5, 1)
action: -0.7, lim=706.15
  volume: 5.0
  limit : 706.15
volume_traded_percent: 0.5 (5.0/10.0) 10.0
('cost', 4.1266098997912195, 0.82532197995824386, 3527.25, 3523.1233901002088)
('new_vol', 0.25, 0.0, -0.75, -4.5)
('cashflow', 5.0, 3527.25, 0.5, 0.25)
0.25
('vol', 0.75, 2)
action: -0.7, lim=706.15
  volume: 7.5
  limit : 706.15
volume_traded_percent: 0.50884558 (5.0884558/10.0) 6.78460773333
('cost', 4.1996144157865274, 0.82532197995834555, 3589.6511441100006, 3585.4515296942141)
('new_vol', 0.25, 0.24115441999999998, -0.52211395, -4.25)
('cashflow', 5.0884558, 3589.6511441100006, 0.75, 0.25)
0.25
('vol', 1.0, 3)
action: -0.7, lim=706.15
  volume: 10.0
  limit : 706.15
volume_traded_percent: 0.5

...

,Amount,Price,Type,norm_Price,Volume,VolumeAcc
989,1.700000,703.360000,bid,0.998205,1195.712000,4910.381485
990,0.001250,703.700000,bid,0.998688,0.879625,3714.669485
991,5.276769,703.800000,bid,0.998830,3713.789860,3713.789860
992,0.000000,704.624678,center,1.000000,0.000000,0.000000
993,5.049153,705.450000,ask,1.001171,3561.924843,3561.924843
994,0.052838,706.170000,ask,1.002193,37.312497,3599.237340
995,0.645323,706.190000,ask,1.002221,455.720409,4054.957749


('vol', 0.25, 0)
action: -0.7, lim=706.15
  volume: 2.5
  limit : 706.15
volume_traded_percent: 0.25 (2.5/10.0) 10.0
('cost', 2.0633049498956098, 0.82532197995824386, 1763.625, 1761.5616950501044)
('new_vol', 0.25, 0.0, -0.375, -2.25)
('cashflow', 2.5, 1763.625, 0.25, 0.25)
0.25
('vol', 0.5, 1)
action: -0.7, lim=706.15
  volume: 5.0
  limit : 706.15
volume_traded_percent: 0.5 (5.0/10.0) 10.0
('cost', 4.1266098997912195, 0.82532197995824386, 3527.25, 3523.1233901002088)
('new_vol', 0.25, 0.0, -0.75, -4.5)
('cashflow', 5.0, 3527.25, 0.5, 0.25)
0.25
('vol', 0.75, 2)
action: -0.7, lim=706.15
  volume: 7.5
  limit : 706.15
volume_traded_percent: 0.50491528 (5.0491528/10.0) 6.73220373333
('cost', 4.1671767860079854, 0.82532197995829826, 3561.92484276, 3557.7576659739921)
('new_vol', 0.25, 0.24508472000000003, -0.5122882, -4.25)
('cashflow', 5.0491528, 3561.92484276, 0.75, 0.25)
0.25
('vol', 1.0, 3)
action: -0.7, lim=706.15
  volume: 10.0
  limit : 706.15
volume_traded_percent: 0.50491528 (5.

,Amount,Price,Type,norm_Price,Volume,VolumeAcc
989,1.461657,703.500000,bid,0.997554,1028.275411,1140.708568
990,0.001250,703.700000,bid,0.997838,0.879625,112.433157
991,0.158232,705.000000,bid,0.999681,111.553532,111.553532
992,0.000000,705.224976,center,1.000000,0.000000,0.000000
993,3.172181,705.450000,ask,1.000319,2237.815171,2237.815171
994,0.052838,706.170000,ask,1.001340,37.312497,2275.127669
995,0.645323,706.190000,ask,1.001368,455.720409,2730.848078


('vol', 0.25, 0)
action: -0.7, lim=706.15
  volume: 2.5
  limit : 706.15
volume_traded_percent: 0.25 (2.5/10.0) 10.0
('cost', 0.56255982041807329, 0.22502392816722933, 1763.625, 1763.0624401795819)
('new_vol', 0.25, 0.0, -0.375, -2.25)
('cashflow', 2.5, 1763.625, 0.25, 0.25)
0.25
('vol', 0.5, 1)
action: -0.7, lim=706.15
  volume: 5.0
  limit : 706.15
volume_traded_percent: 0.317218112 (3.17218112/10.0) 6.34436224
('cost', 0.7138166564805033, 0.22502392816728678, 2237.815171104, 2237.1013544475195)
('new_vol', 0.25, 0.182781888, -0.29304527999999996, -2.5)
('cashflow', 3.17218112, 2237.815171104, 0.5, 0.25)
0.25
('vol', 0.75, 2)
action: -0.7, lim=706.15
  volume: 7.5
  limit : 706.15
volume_traded_percent: 0.317218112 (3.17218112/10.0) 4.22957482667
('cost', 0.7138166564805033, 0.22502392816728678, 2237.815171104, 2237.1013544475195)
('new_vol', 0.5, 0.432781888, -0.043045279999999964, -2.25)
('cashflow', 3.17218112, 2237.815171104, 0.75, 0.5)
0.5
('vol', 1.0, 3)
action: -0.7, lim=706.1

,0
"(t0, i0)",0.112512
"(t0, i1)",0.142763
"(t0, i2)",0.142763
"(t0, i3)",0.142763
"(t1, i0)",0.515826
"(t1, i1)",1.031652
"(t1, i2)",1.041794
"(t1, i3)",1.041794
"(t2, i0)",0.687768
"(t2, i1)",1.375537


In [ ]:
l = [0.1, .2, .3]
print(type(l))
l.index(.2)

In [ ]:
orderbooks_window = [orderbooks[idx*10] for idx in range(H)]
timestamps_window = [timestamps[idx*10] for idx in range(H)]

max_steps = 10
epochs = 1

n_step = 5
replay = Memory(bufferSize)
output = Memory(5)
# stores tuples of (S, A, R, S')

acc_reward = 0
for i_episode in range(epochs):
    state = env.reset()
    # state = discretize_state(state)
    
    acc_reward = 0
    for step in range(max_steps)[::-1]:

        qval = model.predict(state.reshape(1, STATE_DIM))

        if random.random() < epsilon:
            # choose random action
            action = env.action_space.sample()
        else:
            # choose best action from Q(s,a) values
            action = np.argmax(qval)

        
        new_state, reward, done, info = env.step(action)
        # new_state = discretize_state(new_state)

        if done:
            if abs(new_state[0]) > 2.4:
                reward = -1000
            else:
                reward = -100
#             reward = -300 
        else:
            if 0.5 < new_state[0] < 1.5:
                reward = 5
#            reward = 4 - (2* (new_state[0] + 1.)**2)
            # print(reward)
        acc_reward += reward
        
        replay.add((state, action, reward, new_state))
        if (replay.size() >= bufferSize):
            # wait for buffer to be filled, before getting started with training
            minibatch = replay.get_random_samples(batchSize)           
            
            X_train = []
            y_train = []
            for memory in minibatch:
                state_m, action_m, reward_m, new_state_m = memory
                
                qval_old = model.predict(state_m.reshape(1, STATE_DIM))               
                y = np.zeros((1, NUM_ACTIONS))
                y[:] = qval_old[:]
                
                qval_new_m = model.predict(new_state_m.reshape(1, STATE_DIM), batch_size=1)
                maxQ = np.max(qval_new_m)
                
                update = reward_m + (gamma*maxQ)
                y[0][action_m] = update  # target output
                
                X_train.append(state_m.reshape(STATE_DIM,))
                y_train.append(y.reshape(NUM_ACTIONS,))
            X_train = np.array(X_train)
            y_train = np.array(y_train)
                        
            # print("Game #: %s" % (i_episode,))
            model.fit(X_train, y_train, batch_size=batchSize, nb_epoch=1, verbose=0)
        state = new_state
        if done:
            break
       
    # reduce exploration rate
    if epsilon > MIN_EXPLORATION_RATE:
        epsilon = MAX_EXPLORATION_RATE *   math.exp(- DECAY_RATE * i_episode)
    info = "{:4d}/{}: epsilon={:5.3f}, reward_acc: {:0.5f}, last_reward {}, steps: {}"\
            .format(i_episode, epochs, epsilon,  acc_reward, reward, step)
    # output.add(info)
    # clear_output(wait=True)    
    # for x in output.content:
    #     print(x)
    print(info)
    # if epsilon > 0:
    #     epsilon -= (1.5/epochs)
        